# SWE3011_41 Task2


**Prompt Engineering with Langchain for LLMS**

1. Based on the given code, you need to extend (modify) to a classification task using Langchain.
2. Since large language model baseline is required for the task, you may choose any model from OpenAI API and HuggingfaceHub.

 **OpenAI**: Only available for paid users.

 **HuggingfaceHUB**: Free to use with usage limits (reset hourly).

3. Conduct experiments and document the results in the report. Here, you should consider what kind of prompt design you use so please find some tutorials/resources in our homework description to obtain more information.


**0. Installation**

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [44]:
from getpass import getpass
import os

# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token
HUGGINGFACEHUB_API_TOKEN = getpass()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


··········


In [ ]:
pip install huggingface_hub

In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00


**1. Load Dataset**


Evaluation should be done using **provided test dataset**

In [ ]:
from datasets import load_dataset
import pandas as pd

# You can use train_ds for few-shot examples
train_ds = load_dataset("glue", "sst2", split="train")

# Convert the dataset to a Pandas DataFrame
df_train = train_ds.to_pandas()

# Display the DataFrame
print(df_train.head())

# Evaluation should be done using test_ds
test_ds = load_dataset("csv", data_files="./test_dataset.csv")['train']

# Load the dataset from CSV
dataset_path = "/content/test_dataset.csv"  # Replace with the actual path to your CSV file
df_test = pd.read_csv(dataset_path)

# Display the DataFrame
print(df_test.head())

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

                                            sentence  label  idx
0       hide new secretions from the parental units       0    0
1               contains no wit , only labored gags       0    1
2  that loves its characters and communicates som...      1    2
3  remains utterly satisfied to remain the same t...      0    3
4  on the worst revenge-of-the-nerds clichés the ...      0    4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

                                            sentence  label
0  the result is a gaudy bag of stale candy , som...      0
1  if your taste runs to ` difficult ' films you ...      1
2          no way i can believe this load of junk .       0
3  it appears that something has been lost in the...      0
4  it all feels like a monty python sketch gone h...      0


In [ ]:
import random
import numpy as np

def gen_data(df, flag=True, samples=27):
    examples = []

    n = len(df) if flag else samples

    # Count occurrences of each category
    category_counts = df["label"].value_counts()

    for i in range(0, n):
        # Choose category proportionally based on occurrences
        category = np.random.choice(category_counts.index.tolist(), p=(category_counts / category_counts.sum()).tolist())

        # Get a random row with the chosen category
        category_df = df[df["label"] == category]
        random_row = category_df.sample()

        obj = {
            "messages": random_row["sentence"].iloc[0],
            "categories": random_row["label"].iloc[0]
        }
        examples.append(obj)

    random.shuffle(examples)

    return examples


**2. Preparing Prompt**

In [ ]:
from langchain import PromptTemplate

prompt_template = '''
Messages: {messages}
Categories: {categories}
'''

example_prompt = PromptTemplate(
    input_variables=["messages", "categories"],
    template=prompt_template)


In [ ]:
import re

def trim(s):
    if "positive" in s.lower():
        return 1
    elif "negative" in s.lower():
        return 0
    else:
        return -1  # or any default value for the case where no match is found


In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

def eval(extraction_chain, example):
    preds = []
    actual = []

    for x in tqdm(example):
        p = extraction_chain.predict(messages=x["messages"])
        p = trim(p)
        a = x["categories"]
        preds.append(p)
        actual.append(a)

    accuracy = accuracy_score(actual, preds)
    return preds, actual, accuracy


In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain

def initialize_llm(model_name, api_key=None):
    """
    Initialize the model using the langchain library.
    """
    llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature": 0.5, "max_length": 64})

    return llm

**Few-Shot**

In [ ]:
from langchain import FewShotPromptTemplate

examples = gen_data(df_train, False)
print(examples)

# Create the FewShotPromptTemplate
final_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Messages: {messages} \nCategories: ",
    input_variables=["messages"],
    prefix="""You are a sentiment classifier. Classify the given text input as "positive" or "negative" based on its tone.
            Please read each sentence carefully and determine its sentiment based on common definitions.

            <positive>: A positive sentiment indicates a favorable or optimistic tone. It expresses approval, satisfaction, or positive emotions.

            <negative>: A negative sentiment indicates an unfavorable or pessimistic tone. It expresses disapproval, dissatisfaction, or negative emotions.

            Please read each sentence and classify it as either "positive" or "negative" based on the provided definitions.
    """
)

# Initialize the language model
model_name = "google/flan-t5-xxl"
llm = initialize_llm(model_name)
val = gen_data(df_test)

# Create the LLMChain
extraction_chain = LLMChain(llm=llm, prompt=final_prompt, output_key="categories")

# Evaluate and print results without a new line after a comma
preds, actual, acc = eval(extraction_chain, val)
print("\nPredicted Categories:", preds)

print("Actual Categories:", actual)

print(f"Accuracy: {acc * 100:.2f}%")


[{'messages': ", despite downplaying her good looks , carries a little too much ai n't - she-cute baggage into her lead role as a troubled and determined homicide cop to quite pull off the heavy stuff . ", 'categories': 0}, {'messages': 'fun and nimble . ', 'categories': 1}, {'messages': "has produced in recent memory , even if it 's far tamer than advertised . ", 'categories': 1}, {'messages': 'on hammy at times ', 'categories': 1}, {'messages': "'s the perfect star vehicle for grant , allowing him to finally move away from his usual bumbling , tongue-tied screen persona ", 'categories': 1}, {'messages': 'is ultimately quite unengaging ', 'categories': 0}, {'messages': 'veered off too far into the exxon zone , and ', 'categories': 0}, {'messages': 'ends up as tedious as the chatter of parrots raised on oprah ', 'categories': 0}, {'messages': 'exasperated ', 'categories': 0}, {'messages': 'breathless anticipation ', 'categories': 1}, {'messages': 'but kouyate elicits strong performance

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
100%|██████████| 100/100 [01:07<00:00,  1.49it/s]


Predicted Categories: [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, -1, 1, 0, 0, 0, 0, 0, 1, -1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]
Actual Categories: [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]
Accuracy: 83.00%


In [39]:
from langchain import FewShotPromptTemplate

# Initialize the language model
model_name = "google/flan-t5-xxl"
llm = initialize_llm(model_name)

# List to store accuracies
accuracies = []

# Perform 10 iterations
for iteration in range(1, 4):
    print(f"\nIteration {iteration}:")

    # Generate new examples for each iteration
    examples = gen_data(df_train, flag = False, samples=27)

    # Define your FewShotPromptTemplate
    final_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        suffix="Messages: {messages} \nCategories: ",
        input_variables=["messages"],
        prefix="""You are a sentiment classifier. Classify the given text input as "positive" or "negative" based on its tone.
                Please read each sentence carefully and determine its sentiment based on common definitions.

                <positive>: A positive sentiment indicates a favorable or optimistic tone. It expresses approval, satisfaction, or positive emotions.

                <negative>: A negative sentiment indicates an unfavorable or pessimistic tone. It expresses disapproval, dissatisfaction, or negative emotions.

                Please read each sentence and classify it as either "positive" or "negative" based on the provided definitions.
        """
    )

    # Create the LLMChain with the updated prompt
    extraction_chain = LLMChain(llm=llm, prompt=final_prompt, output_key="categories")

    # Evaluate and print results without a new line after a comma
    preds, actual, acc = eval(extraction_chain, val)
    accuracies.append(acc)  # Store accuracy in the list
    print("Predicted Categories:", preds)

    print("Actual Categories:", actual)

    print(f"Accuracy: {acc * 100:.2f}%")

# Print all accuracies at the end
print("\nAll Accuracies:", accuracies)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)



Iteration 1:


100%|██████████| 100/100 [00:28<00:00,  3.56it/s]


Predicted Categories: [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Actual Categories: [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]
Accuracy: 89.00%

Iteration 2:


100%|██████████| 100/100 [00:28<00:00,  3.49it/s]


Predicted Categories: [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, -1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Actual Categories: [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]
Accuracy: 87.00%

Iteration 3:


100%|██████████| 100/100 [00:31<00:00,  3.13it/s]

Predicted Categories: [0, 1, 0, 0, 1, 1, 1, -1, 1, -1, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Actual Categories: [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]
Accuracy: 82.00%

All Accuracies: [0.89, 0.87, 0.82]


**Zero-Shot**

In [42]:
from langchain import PromptTemplate

# Create the zero-shot prompt template
zero_shot_template = PromptTemplate(
    input_variables=["messages"],
    template="""You are a sentiment classifier. Classify the given text input as "positive" or "negative" based on its tone.
                Please read each sentence carefully and determine its sentiment based on common definitions.

                <positive>: A positive sentiment indicates a favorable or optimistic tone. It expresses approval, satisfaction, or positive emotions.

                <negative>: A negative sentiment indicates an unfavorable or pessimistic tone. It expresses disapproval, dissatisfaction, or negative emotions.

                Please read each sentence and classify it as either "positive" or "negative" based on the provided definitions.
    """
)

# Initialize the language model
model_name = "google/flan-t5-xxl"
llm = initialize_llm(model_name)
val = gen_data(df_test)

# Create the LLMChain
extraction_chain = LLMChain(llm=llm, prompt=zero_shot_template, output_key="categories")

# Evaluate and print results
preds, actual, acc = eval(extraction_chain, val)
print("\nPredicted Categories:", preds)

print("Actual Categories:", actual)

print(f"Accuracy: {acc * 100:.2f}%", end=" ")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
100%|██████████| 100/100 [00:07<00:00, 14.10it/s]


Predicted Categories: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Actual Categories: [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
Accuracy: 51.00% 